## Basic Connection

In [1]:
import os
import requests
from notion_client import Client
os.environ['NOTION_API_KEY'] = "secret_jnUpyC7BqRV1CEF3LmeEJ2sQPSFqKuiWsWtdnV2KIER"

notion = Client(auth=os.environ["NOTION_API_KEY"])

headers = {
    "Authorization": f"{os.environ['NOTION_API_KEY']}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

## Database 객체에 있는 페이지들의 ID 리스트 반환

In [87]:

# 데이터베이스 ID 및 Notion API URL
database_id = "907646de285c46b6bfdca72b26bc6e47"
notion_api_url = f"https://api.notion.com/v1/databases/{database_id}/query"

# 페이지 요청 (필터나 정렬이 없는 기본 쿼리)
response = requests.post(notion_api_url, headers=headers, json={})
# 응답 처리
if response.status_code == 200:
    pages = response.json().get('results', [])
    print("Page IDs in the database:")
    for page in pages:
        page_id = page.get('id')
        page_title = page.get('properties').get('Name').get('title')[0].get('plain_text')
        page_created_at = page.get('created_time')
        print(f"{page_title:30}\t{page_created_at}\t{page_id}")
else:
    print("Error:", response.status_code, response.json())

Page IDs in the database:
ws_album_import               	2024-08-28T02:18:00.000Z	61be24b4-f4c8-41bd-bcf2-e30cb300c3c7
ws_album                      	2024-08-28T00:43:00.000Z	618b7a7d-0b74-4cf3-8457-41ee944c0562
we_meta.we_digital_product    	2024-08-20T06:56:00.000Z	02299f5a-f8c9-4b04-85a6-5fc634fc7ab9
we_meta.we_media_product      	2024-08-20T06:55:00.000Z	4663eb83-f394-45de-b453-4d6ee08e6c95
we_meta.ws_album_legacy       	2024-08-20T02:43:00.000Z	a83ff114-8349-4dd5-a9ac-c838d58f56be


## Page에 Block 생성

In [26]:
PAGE_ID = "16beff63621f4cd3a66ceee5726e2dc4".replace("-", "")
url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"
type = "divider"
data = {
    "children": [
        {
            "object": "block",
            "type": "table",
            "table": {
                "table_width": 2,  # 테이블 열 개수
                "has_column_header": True,  # 첫 번째 행을 헤더로 사용할지 여부
                "has_row_header": False,  # 첫 번째 열을 헤더로 사용할지 여부
                "children": [
                    {
                        "object": "block",
                        "type": "table_row",
                        "table_row": {
                            "cells": [
                                [{"type": "text", "text": {"content": "Header 1"}}],
                                [{"type": "text", "text": {"content": "Header 2"}}]
                            ]
                        }
                    },
                    {
                        "object": "block",
                        "type": "table_row",
                        "table_row": {
                            "cells": [
                                [{"type": "text", "text": {"content": "Row 1, Col 1"}}],
                                [{"type": "text", "text": {"content": "Row 1, Col 2"}}]
                            ]
                        }
                    },
                    {
                        "object": "block",
                        "type": "table_row",
                        "table_row": {
                            "cells": [
                                [{"type": "text", "text": {"content": "Row 2, Col 1"}}],
                                [{"type": "text", "text": {"content": "Row 2, Col 2"}}]
                            ]
                        }
                    }
                ]
            }
        }
    ]
}

response = requests.patch(url, headers=headers, json=data)

# 응답 확인
if response.status_code == 200:
    print("블록이 성공적으로 추가되었습니다!")
else:
    print(f"추가 실패: {response.status_code}, {response.json()}")

블록이 성공적으로 추가되었습니다!


## MD 파일을 그대로 페이지에 올리기 Using MarkdownIt

In [32]:
from markdown_it import MarkdownIt

PAGE_ID = "618b7a7d-0b74-4cf3-8457-41ee944c0562".replace("-", "")
markdown_file_path = "/Users/lymansong/Documents/GitHub/mtms/data/specs/ws_album.md"
with open(markdown_file_path, 'r') as file:
    markdown_content = file.read()

# Markdown을 Notion 블록 형식으로 변환
md = MarkdownIt()
tokens = md.parse(markdown_content)

In [34]:
def convert_md_to_notion_blocks(tokens):
    blocks = []
    for token in tokens:
        if token.type == "heading_open":
            level = int(token.tag[1])
            content = tokens[tokens.index(token) + 1].content
            blocks.append({
                "object": "block",
                "type": f"heading_{level}",
                f"heading_{level}": {
                    "rich_text": [{"type": "text", "text": {"content": content}}]
                }
            })
        elif token.type == "paragraph_open":
            content = tokens[tokens.index(token) + 1].content
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": content}}]
                }
            })
        # 여기에 추가적인 Markdown 형식을 다룰 수 있습니다.
    return blocks

notion_blocks = convert_md_to_notion_blocks(tokens)

## 블록 가져오기

In [3]:
PAGE_ID = "16beff63621f4cd3a66ceee5726e2dc4".replace("-", "")
url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"

response = requests.get(url, headers=headers)

# 응답 확인 및 블록 출력
if response.status_code == 200:
    blocks = response.json().get("results", [])
    for block in blocks:
        print(block)
else:
    print(f"블록을 가져오지 못했습니다: {response.status_code}, {response.json()}")


{'object': 'block', 'id': 'e6cdfe1c-c009-4025-acca-9c21eaaa4da4', 'parent': {'type': 'page_id', 'page_id': '16beff63-621f-4cd3-a66c-eee5726e2dc4'}, 'created_time': '2024-09-06T06:00:00.000Z', 'last_edited_time': '2024-09-06T06:00:00.000Z', 'created_by': {'object': 'user', 'id': '73c553bc-accf-4140-9556-edf439cc245e'}, 'last_edited_by': {'object': 'user', 'id': '73c553bc-accf-4140-9556-edf439cc245e'}, 'has_children': False, 'archived': False, 'in_trash': False, 'type': 'heading_1', 'heading_1': {'rich_text': [{'type': 'text', 'text': {'content': 'we_mart.ws_album_sale', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'we_mart.ws_album_sale', 'href': None}], 'is_toggleable': False, 'color': 'default'}}
{'object': 'block', 'id': '3d0eff92-87fe-4f96-b158-1abb172b97bd', 'parent': {'type': 'page_id', 'page_id': '16beff63-621f-4cd3-a66c-eee5726e2dc4'}, 'created_time': '2024-09-06T06:00

### 테이블 row 가져오기

In [45]:
def get_table_rows(table_block_id):
    url = f"https://api.notion.com/v1/blocks/{table_block_id}/children"
    rows = []
    has_more = True
    start_cursor = None

    while has_more:
        response = requests.get(url, headers=headers, params={"start_cursor": start_cursor} if start_cursor else {})
        if response.status_code == 200:
            data = response.json()
            rows.extend(data.get('results', []))
            has_more = data.get('has_more', False)
            start_cursor = data.get('next_cursor', None)
        else:
            print(f"테이블 행을 가져오지 못했습니다: {response.status_code}, {response.json()}")
            has_more = False

    return rows
rows = get_table_rows(table_block_id="dc1922a8-0816-4f94-b22b-301cd82a47ab".replace("-", ""))
print(rows[0])

{'object': 'block', 'id': 'be34b529-610b-4010-b774-9ef789e4d047', 'parent': {'type': 'block_id', 'block_id': 'dc1922a8-0816-4f94-b22b-301cd82a47ab'}, 'created_time': '2024-08-28T02:18:00.000Z', 'last_edited_time': '2024-08-28T02:18:00.000Z', 'created_by': {'object': 'user', 'id': '44ff2ed5-55be-46c7-b73f-bb5823e4c641'}, 'last_edited_by': {'object': 'user', 'id': '44ff2ed5-55be-46c7-b73f-bb5823e4c641'}, 'has_children': False, 'archived': False, 'in_trash': False, 'type': 'table_row', 'table_row': {'cells': [[{'type': 'text', 'text': {'content': 'About', 'link': None}, 'annotations': {'bold': True, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'About', 'href': None}], [{'type': 'text', 'text': {'content': '위버스샵 판매 앨범 메타 정보', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': '위버스샵 판매 앨범 메타 정보', 'href': None}]]}}


## 페이지에 블록 추가하기

In [43]:
# Notion 페이지에 블록 추가 요청
PAGE_ID = "16beff63621f4cd3a66ceee5726e2dc4".replace("-", "")
url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"
for block in blocks:
    response = requests.patch(url, headers=headers, json={"children": [block]})
    if response.status_code == 200:
        print("블록이 성공적으로 추가되었습니다!")
    else:
        print(f"추가 실패: {response.status_code}, {response.json()}")

블록이 성공적으로 추가되었습니다!
추가 실패: 400, {'object': 'error', 'status': 400, 'code': 'validation_error', 'message': 'body failed validation: body.children[0].table.children should be defined, instead was `undefined`.', 'request_id': '03492cf9-f031-4659-8e56-3689ee9636d2'}
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
추가 실패: 400, {'object': 'error', 'status': 400, 'code': 'validation_error', 'message': 'body failed validation: body.children[0].table.children should be defined, instead was `undefined`.', 'request_id': '1471d653-bb29-4666-850f-2fdab4c37f39'}
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
블록이 성공적으로 추가되었습니다!
추가 실패: 400, {'object': 'error', 'status': 400, 'code': 'validation_error', 'message': 'body failed validation: body

### 테이블 블록 추가

In [90]:
# 테이블 블록 추가 함수
def add_table_block(page_id, table_block):
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    
    
    # POST 요청으로 테이블 블록 추가
    response = requests.patch(url, headers=headers, json={"children": [table_block]})

    if response.status_code == 200:
        table_id = response.json()['results'][0]['id']
        print("테이블 블록이 성공적으로 추가되었습니다.")
        return table_id
    else:
        print(f"테이블 블록 추가 실패: {response.status_code}, {response.json()}")
        return None

# 테이블 행 블록 추가 함수
def add_table_rows(table_block_id, rows):
    url = f"https://api.notion.com/v1/blocks/{table_block_id}/children"
    
    # 테이블 행 블록들 정의
    table_row_blocks = []
    for row in rows:
        table_row_blocks.append({
            "object": "block",
            "type": "table_row",
            "table_row": {
                "cells": [[{"type": "text", "text": {"content": cell}}] for cell in row]
            }
        })

    # PATCH 요청으로 테이블 행 블록 추가
    response = requests.patch(url, headers=headers, json={"children": table_row_blocks})

    if response.status_code == 200:
        print("테이블 행이 성공적으로 추가되었습니다.")
    else:
        print(f"테이블 행 추가 실패: {response.status_code}, {response.json()}")

In [96]:
blocks[1]

{'object': 'block',
 'id': 'dc1922a8-0816-4f94-b22b-301cd82a47ab',
 'parent': {'type': 'page_id',
  'page_id': '61be24b4-f4c8-41bd-bcf2-e30cb300c3c7'},
 'created_time': '2024-08-28T02:18:00.000Z',
 'last_edited_time': '2024-08-28T02:18:00.000Z',
 'created_by': {'object': 'user',
  'id': '44ff2ed5-55be-46c7-b73f-bb5823e4c641'},
 'last_edited_by': {'object': 'user',
  'id': '44ff2ed5-55be-46c7-b73f-bb5823e4c641'},
 'has_children': True,
 'archived': False,
 'in_trash': False,
 'type': 'table',
 'table': {'table_width': 2,
  'has_column_header': False,
  'has_row_header': False}}

In [137]:
table_sample = {
   "children":[
      {
         "object":"block",
         "type":"table",
         "table":{
            "table_width":2,
            "has_column_header":False,
            "has_row_header":False,
            "children":[
               {
                  "object":"block",
                  "type":"table_row",
                  "table_row":{
                     "cells":[
                        [
                           {
                              "type":"text",
                              "text":{
                                 "content":"About",
                                 "link":None
                              },
                              "annotations":{
                                 "bold":True,
                                 "italic":False,
                                 "strikethrough":False,
                                 "underline":False,
                                 "code":False,
                                 "color":"default"
                              },
                              "plain_text":"About",
                              "href":None
                           }
                        ],
                        [
                           {
                              "type":"text",
                              "text":{
                                 "content":"위버스샵 판매 앨범 메타 정보",
                                 "link":None
                              },
                              "annotations":{
                                 "bold":False,
                                 "italic":False,
                                 "strikethrough":False,
                                 "underline":False,
                                 "code":False,
                                 "color":"default"
                              },
                              "plain_text":"위버스샵 판매 앨범 메타 정보",
                              "href":None
                           }
                        ]
                     ]
                  }
               }
            ]
         }
      }
   ]
}

In [150]:
from typing import List, Dict, Tuple
def get_a_table_row(content_list:List[Tuple]) -> Dict:
    cell_list = []
    for content in content_list:
        cell_list.append(
            [
                {
                    "type":"text",
                    "text":{
                        "content":content[0],
                        "link":None
                    },
                    "annotations":content[1],
                    "plain_text":content[0],
                    "href":None
                }
            ]
        )
    row = {
        "object" : "block",
        "type" : "table_row",
        "table_row" : {"cells" : cell_list}
    }
    return row
def get_a_table(table_width, has_column_header, has_row_header, table_rows:List[Dict]):
    table_obj = {
        "table_width":table_width,
        "has_column_header":has_column_header,
        "has_row_header":has_row_header,
    }
    table_obj['children'] = table_rows

    table = {
        "object" : "block",
        "type" : "table",
        "table" : table_obj
    }

    return {"children" : [table]}

In [153]:
row

NameError: name 'row' is not defined

In [156]:
c_list = []
c_list.append(
    (
        "Content",
        {
            "bold":True,
            "italic":False,
            "strikethrough":False,
            "underline":False,
            "code":False,
            "color":"default"
        }
    )
)
c_list.append(
    (
        "Sample",
        {
            "bold":False,
            "italic":False,
            "strikethrough":False,
            "underline":False,
            "code":False,
            "color":"default"
        }
    )
)
row = get_a_table_row(c_list)
table = get_a_table(len(c_list), False, False, row)
table

{'children': [{'object': 'block',
   'type': 'table',
   'table': {'table_width': 2,
    'has_column_header': False,
    'has_row_header': False,
    'children': [{'object': 'block',
      'type': 'table_row',
      'table_row': {'cells': [[{'type': 'text',
          'text': {'content': 'Content', 'link': None},
          'annotations': {'bold': True,
           'italic': False,
           'strikethrough': False,
           'underline': False,
           'code': False,
           'color': 'default'},
          'plain_text': 'Content',
          'href': None}],
        [{'type': 'text',
          'text': {'content': 'Sample', 'link': None},
          'annotations': {'bold': False,
           'italic': False,
           'strikethrough': False,
           'underline': False,
           'code': False,
           'color': 'default'},
          'plain_text': 'Sample',
          'href': None}]]}}]}}]}

In [157]:
PAGE_ID = "618b7a7d-0b74-4cf3-8457-41ee944c0562".replace("-", "")
url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"
response = requests.patch(url, headers=headers, json=table)

# 응답 확인
if response.status_code == 200:
    print("테이블이 성공적으로 추가되었습니다!")
else:
    print(f"추가 실패: {response.status_code}, {response.json()}")


테이블이 성공적으로 추가되었습니다!


In [79]:
{"object" : rows[0]['object'], "type" : rows[0]['type'], "table_row" : rows[0]['table_row']}

{'object': 'block',
 'type': 'table_row',
 'table_row': {'cells': [[{'type': 'text',
     'text': {'content': 'About', 'link': None},
     'annotations': {'bold': True,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'About',
     'href': None}],
   [{'type': 'text',
     'text': {'content': '위버스샵 판매 앨범 메타 정보', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': '위버스샵 판매 앨범 메타 정보',
     'href': None}]]}}

In [75]:
cur_row

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [5]:
from parser_utils import get_text_block

get_text_block(content_string = 'Change History', block_type = 'toggle')

{'object': 'block',
 'type': 'toggle',
 'toggle': {'rich_text': [{'type': 'text',
    'text': {'content': 'Change History', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': 'Change History',
    'href': None}]}}